In [ ]:
!pip install simpletransformers csrgraph networkx==2.8.8 --upgrade

In [ ]:
from gpt import GPT
from hpc import HPC
from statistical_learning_utils import get_sequence
from graph_utils import get_graph, get_walks_as_strings, plot_path
from story_utils import get_stories
from tqdm import tqdm
import random
import pandas as pd
import torch
from random import shuffle

#### Test with spatial trajectories

In [ ]:
seqs = get_walks_as_strings()
len(seqs)

In [ ]:
seqs[0]

In [ ]:
testgpt = GPT(vocab_size=100)
testhpc = HPC()

# testhpc.encode(seqs)
# seqs = [testhpc.replay() for i in tqdm(range(10000))]

In [ ]:
testgpt.train(seqs[0:5000], best_model_dir='outputs_graph')

In [ ]:
testgpt = GPT(base_model='outputs_graph', base_model_name='gpt2')

In [ ]:
testgpt.continue_input('v N y S', do_sample=False)

#### Test with Durrant sequences

In [ ]:
testgpt = GPT(vocab_size=10)
testhpc = HPC()

In [ ]:
seqs = [get_sequence() for i in range(5000)]
# testhpc.encode(seqs)
# seqs = [testhpc.replay() for i in range(10000)]

In [ ]:
testgpt.train(seqs[0:5000], best_model_dir='outputs_durrant')

In [ ]:
testgpt = GPT(base_model='outputs_durrant', base_model_name='gpt2')

In [ ]:
testgpt.continue_input('1,5,', do_sample=False)

#### Test with cloze stories

In [ ]:
df = pd.read_csv('data/stories_train.csv')
df["combined"] = df["sentence1"].str.cat(df[[f"sentence{i}" for i in range(2,5)]].astype(str), sep=" ")

In [ ]:
def get_random_sentence():
    random_story = random.choice(df["combined"].tolist())
    return " " + random_story[0:random_story.index('.')]

typical = []
atypical_short = []
atypical_long = []

sents_list = []
num_typical = 1000

for story in df["combined"].tolist()[0:num_typical]:
    typical.append(story + " END ")

for story in df["combined"].tolist()[num_typical:num_typical+10]:
    story = '.'.join(story.split('.')[0:3]) + " END "
    print(story)
    print("...........")
    atypical_short.append(story)
    
for story in df["combined"].tolist()[num_typical+10:num_typical+20]:
    story = story + get_random_sentence() + " END "
    print(story)
    print("...........")
    atypical_long.append(story)

sents_list = typical + atypical_short + atypical_long
shuffle(sents_list)

In [ ]:
# seqs = get_stories()

# testgpt = GPT(base_model='gpt2-large')
# testhpc = HPC()

# testhpc.encode(seqs)
# seqs = [testhpc.replay() for i in range(5000)]

# testgpt.train(random.sample(seqs, 20000), best_model_dir='outputs_stories')

torch.cuda.empty_cache()

testgpt = GPT(base_model='gpt2-medium')
testhpc = HPC()

testgpt.train(sents_list, best_model_dir='outputs_stories', eps=50)

In [ ]:
testgpt = GPT(base_model='outputs_stories', base_model_name='gpt2-medium')

In [ ]:
out = testgpt.continue_input('Bob grew up watching boxing on TV.', do_sample=False)
print(out)

In [ ]:
for s in typical:
    print(s)
    out = testgpt.continue_input(s.split('.')[0], do_sample=False, max_length=100)